In [21]:
#documentation: https://librosa.github.io/librosa/generated/librosa.feature.mfcc.html

from python_speech_features import mfcc
from python_speech_features import logfbank
import scipy.io.wavfile as wav
import glob
import sys
import numpy as np


In [33]:
import os

if os.getcwd().endswith("Calculator") == False and \
   os.getcwd().endswith("Calculator\\") == False and \
   os.getcwd().endswith("Calculator/") == False :
    cwd = os.getcwd()

print(cwd)

C:\Users\qan96\Documents\GitHub\Audio-Calculator
C:\Users\qan96\Documents\GitHub\Audio-Calculator\recording data\five


In [32]:
def join_features(mfcc, fbank):
    features = np.concatenate((mfcc, fbank), axis=1)
    return features

features = []
labels = []

def read_file(files, label):
    for file in files:
        (rate,sig) = wav.read(file)
        mfcc_feat = mfcc(sig, rate, nfft=1536)
        fbank_feat = logfbank(sig, rate, nfft=1536)
        acoustic_features = join_features(mfcc_feat, fbank_feat)
        features.append(acoustic_features)
        labels.append(label)
        
feature_types = ["zero", "one", "two", "three", "four", "five", "six", "seven", "eight",\
                 "nine", "plus", "minus", "times", "over"]

for i in feature_types:
    if sys.platform.startswith('win32'):
         os.chdir(cwd+"\\recording data\\" + i + "\\")
    elif sys.platform.startswith('darwin'):
         os.chdir(cwd+"/recording data/" + i + "/")
       
    files = glob.glob( os.path.join(os.getcwd(), '*.wav') )
    print(files)
    read_file(files, i)
    

['C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\zero\\0_jackson_0.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\zero\\0_jackson_1.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\zero\\0_jackson_10.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\zero\\0_jackson_11.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\zero\\0_jackson_12.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\zero\\0_jackson_13.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\zero\\0_jackson_14.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\zero\\0_jackson_15.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\zero\\0_jackson_16.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\zero\\0_jackson_17.wav', 'C:\\Users\\qan96\\Documents\\G

['C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\one\\1_jackson_0.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\one\\1_jackson_1.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\one\\1_jackson_10.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\one\\1_jackson_11.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\one\\1_jackson_12.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\one\\1_jackson_13.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\one\\1_jackson_14.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\one\\1_jackson_15.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\one\\1_jackson_16.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\one\\1_jackson_17.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Aud

['C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\two\\2_jackson_0.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\two\\2_jackson_1.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\two\\2_jackson_10.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\two\\2_jackson_11.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\two\\2_jackson_12.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\two\\2_jackson_13.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\two\\2_jackson_14.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\two\\2_jackson_15.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\two\\2_jackson_16.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\two\\2_jackson_17.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Aud

['C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\three\\3_jackson_0.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\three\\3_jackson_1.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\three\\3_jackson_10.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\three\\3_jackson_11.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\three\\3_jackson_12.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\three\\3_jackson_13.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\three\\3_jackson_14.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\three\\3_jackson_15.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\three\\3_jackson_16.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\three\\3_jackson_17.wav', 'C:\\Users\\qan96\\Do

['C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\four\\4_jackson_0.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\four\\4_jackson_1.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\four\\4_jackson_10.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\four\\4_jackson_11.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\four\\4_jackson_12.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\four\\4_jackson_13.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\four\\4_jackson_14.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\four\\4_jackson_15.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\four\\4_jackson_16.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\four\\4_jackson_17.wav', 'C:\\Users\\qan96\\Documents\\G

['C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\five\\5_jackson_0.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\five\\5_jackson_1.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\five\\5_jackson_10.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\five\\5_jackson_11.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\five\\5_jackson_12.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\five\\5_jackson_13.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\five\\5_jackson_14.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\five\\5_jackson_15.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\five\\5_jackson_16.wav', 'C:\\Users\\qan96\\Documents\\GitHub\\Audio-Calculator\\recording data\\five\\5_jackson_17.wav', 'C:\\Users\\qan96\\Documents\\G

KeyboardInterrupt: 

In [7]:
print(features[1])

[[16.26976594  6.75409837  9.39135608 ... 10.72678368 11.60265751
  12.26555555]
 [17.28148868  7.17576756 12.95945248 ... 11.03262235 12.23517096
  12.61697634]
 [17.61772288  5.64719229 21.7397422  ... 11.68143268 12.53069515
  12.87645569]
 ...
 [13.25928728 13.48520309 -5.18970101 ...  6.92544961  7.63741251
   7.53024637]
 [12.63030516 10.94916611 -4.51168871 ...  6.87031632  7.31165291
   7.53166084]
 [12.06383846 11.25537577  0.50075832 ...  6.22250767  6.92430493
   7.55139646]]


win32
